#Telecom Domain ReadOps Assignment
This notebook contains assignments to practice Spark read options and Databricks volumes. <br>
Sections: Sample data creation, Catalog & Volume creation, Copying data into Volumes, Path glob/recursive reads, toDF() column renaming variants, inferSchema/header/separator experiments, and exercises.<br>

![](https://fplogoimages.withfloats.com/actual/68009c3a43430aff8a30419d.png)
![](https://theciotimes.com/wp-content/uploads/2021/03/TELECOM1.jpg)

##First Import all required libraries & Create spark session object

##1. Write SQL statements to create:
1. A catalog named telecom_catalog_assign
2. A schema landing_zone
3. A volume landing_vol
4. Using dbutils.fs.mkdirs, create folders:<br>
/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/
/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/
/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/
5. Explain the difference between (Just google and understand why we are going for volume concept for prod ready systems):<br>
a. Volume vs DBFS/FileStore<br>
b. Why production teams prefer Volumes for regulated data<br>

In [0]:
%sql
create catalog if not exists telecom_catalog_assign;
create database if not exists telecom_catalog_assign.landing_zone;
create volume if not exists telecom_catalog_assign.landing_zone.landing_vol;


In [0]:
dbutils.fs.mkdirs("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/")
dbutils.fs.mkdirs("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/")
dbutils.fs.mkdirs("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/")


Explain the difference between (Just google and understand why we are going for volume concept for prod ready systems):
a. Volume vs DBFS/FileStore
b. Why production teams prefer Volumes for regulated data

##Data files to use in this usecase:
customer_csv = '''
101,Arun,31,Chennai,PREPAID
102,Meera,45,Bangalore,POSTPAID
103,Irfan,29,Hyderabad,PREPAID
104,Raj,52,Mumbai,POSTPAID
105,,27,Delhi,PREPAID
106,Sneha,abc,Pune,PREPAID
'''

usage_tsv = '''customer_id\tvoice_mins\tdata_mb\tsms_count
101\t320\t1500\t20
102\t120\t4000\t5
103\t540\t600\t52
104\t45\t200\t2
105\t0\t0\t0
'''

tower_logs_region1 = '''event_id|customer_id|tower_id|signal_strength|timestamp
5001|101|TWR01|-80|2025-01-10 10:21:54
5004|104|TWR05|-75|2025-01-10 11:01:12
'''

##2. Filesystem operations
1. Write code to copy the above datasets into your created Volume folders:
Customer → /Volumes/.../customer/
Usage → /Volumes/.../usage/
Tower (region-based) → /Volumes/.../tower/region1/ and /Volumes/.../tower/region2/

2. Write a command to validate whether files were successfully copied

In [0]:
# Input datasets
# -------------------------------
customer_csv = """101,Arun,31,Chennai,PREPAID
102,Meera,45,Bangalore,POSTPAID
103,Irfan,29,Hyderabad,PREPAID
104,Raj,52,Mumbai,POSTPAID
105,,27,Delhi,PREPAID
106,Sneha,abc,Pune,PREPAID
"""

usage_csv = """customer_id\tvoice_mins\tdata_mb\tsms_count
101\t320\t1500\t20
102\t120\t4000\t5
103\t540\t600\t52
104\t45\t200\t2
105\t0\t0\t0
"""

tower_logs_region1 = """event_id|customer_id|tower_id|signal_strength|timestamp
5001|101|TWR01|-80|2025-01-10 10:21:54
5004|104|TWR05|-75|2025-01-10 11:01:12
"""

# Base volume path
base_path = "/Volumes/telecom_catalog_assign/landing_zone/landing_vol/"

customer_path = f"{base_path}/customer"
usage_path = f"{base_path}/usage"
tower_region1_path = f"{base_path}/tower/region1"
tower_region2_path = f"{base_path}/tower/region2"

# Create directories
dbutils.fs.mkdirs(customer_path)
dbutils.fs.mkdirs(usage_path)
dbutils.fs.mkdirs(tower_region1_path)
dbutils.fs.mkdirs(tower_region2_path)

# Write files into Volume folders
dbutils.fs.put(f"{customer_path}/customer.csv", customer_csv, overwrite=True)
dbutils.fs.put(f"{usage_path}/usage.csv", usage_csv, overwrite=True)
dbutils.fs.put(f"{tower_region1_path}/tower_logs_region1.csv", tower_logs_region1, overwrite=True)

print("✅ Files copied successfully into Volume folders")

##3. Directory Read Use Cases
1. Read all tower logs using:
Path glob filter (example: *.csv)
Multiple paths input
Recursive lookup

2. Demonstrate these 3 reads separately:
Using pathGlobFilter
Using list of paths in spark.read.csv([path1, path2])
Using .option("recursiveFileLookup","true")

3. Compare the outputs and understand when each should be used.

##Read all tower logs using: Path glob filter (example: *.csv) Multiple paths input Recursive lookup

In [0]:

from pyspark.sql.types import StructType, StructField, StringType

tower_base_path = "/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower"

schema = StructType([
    StructField("column1", StringType(), True),
    StructField("column2", StringType(), True),
    # Add more fields to match your CSV columns
])

df_glob = (
    spark.read
    .option("header", True)
    .option("pathGlobFilter", "*.csv")
    .schema(schema)
    .csv(tower_base_path)
)

display(df_glob)

Demonstrate these 3 reads separately: Using pathGlobFilter Using list of paths in spark.read.csv([path1, path2]) Using .option("recursiveFileLookup","true")

In [0]:
%python
path1 = "/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region1"
path2 = "/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region2"

df_multi_path = (
    spark.read
    .option("header", True)
    .csv([path1, path2])
)

df_multi_path.show()

##4. Schema Inference, Header, and Separator
1. Try the Customer, Usage files with the option and options using read.csv and format function:<br>
header=false, inferSchema=false<br>
or<br>
header=true, inferSchema=true<br>
2. Write a note on What changed when we use header or inferSchema  with true/false?<br>
3. How schema inference handled “abc” in age?<br>

In [0]:
#header=false, inferSchema=false
customer_path = "/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer"
df_cust_noinfer = (
    spark.read
    .format("csv")
    .option("header", "false")
    .option("inferSchema", "false")
    .load(customer_path)
)

df_cust_noinfer.show()
df_cust_noinfer.printSchema()


#header=true, inferSchema=true

df_cust_infer = (
    spark.read
    .format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load(customer_path)
)

df_cust_infer.show()
df_cust_infer.printSchema()

usage_path="/Volumes/workspace/default/usage_metrics"
df_usage= spark.read.format("csv").option("header","true").option("inferSchema","true").load(usage_path)
df_usage.show()
df_usage.printSchema()


##Note - Header = true ---- Create header names but false doesnt
#inferScheme = true ----detects auto date type but false choose string as default for all

##5. Column Renaming Usecases
1. Apply column names using string using toDF function for customer data
2. Apply column names and datatype using the schema function for usage data
3. Apply column names and datatype using the StructType with IntegerType, StringType, TimestampType and other classes for towers data 

In [0]:
#1.Apply column names using string using toDF function for customer data
customer_path = "/Volumes/workspace/default/volume_wd36/custs"
df1 = spark.read.csv(customer_path).toDF("cust_id","fname","lname","age","profession")
df1.show()
df1.printSchema()

#2.Apply column names and datatype using the schema function for usage data

from pyspark.sql.types import StructType, StructField, IntegerType, StringType

usage_path="/Volumes/workspace/default/usage_metrics"
schema=StructType([
    StructField("date",IntegerType(),True),
    StructField("OS",StringType(),True),
    StructField("Percent of Usage",IntegerType(),True),
 
])
df2=spark.read.csv(usage_path,schema=schema)
df2.show()
df2.printSchema()


#3.Apply column names and datatype using the StructType with IntegerType, StringType, TimestampType and other classes for towers data

from pyspark.sql.types import StructType, StructField, IntegerType, StringType, TimestampType

tower_base_path = "/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower"
schema = StructType([
    StructField("event_id", StringType(), True),
    StructField("customer_id", StringType(), True),
    StructField("tower_id", StringType(), True),
    StructField("signal_strength", StringType(), True),
    StructField("timestamp", TimestampType(), True),
])
df3 = (
    spark.read
    .option("header", True)
    .schema(schema)
    .csv(tower_base_path)
)
df3.show()
df3.printSchema()



## 6. More to come (stay motivated)....